### Esta libreta se utiliza para generar la información necesaria para las Listas de valoresde la aplicación de mapas de Streamlit

In [15]:
import geopandas as gpd
import pandas as pd
import json

In [16]:
data_folder ="../../project/Files/"

In [17]:
def build_dates_LOV():
    # Define the file path
    csv_accidents_path = f'{data_folder}2023_Accidentalidad_c_g_s_d_clean.csv'
    
    dates_list = (
        pd.read_csv(csv_accidents_path , sep = ";")
        .assign(fecha=lambda x: pd.to_datetime(x['fecha'], format='%Y-%m-%d'))
        .assign(fecha_display=lambda x: x['fecha'].dt.strftime('%B-%Y'))
        .groupby('fecha_display')['fecha'].agg(['min' , 'max']).reset_index()
        .astype({'min': str, 'max': str})
        .sort_values('min')
        .to_dict(orient='records')
    )   
    
    return dates_list

def build_districts_LOV():
    shapefile_path = f"{data_folder}Barrios/Barrios.shp"

    key_cod_dist = 'cod_distr'
    key_nombre= 'nombre'

    dict_list = (
        gpd.read_file(shapefile_path)
        .loc[:, ['CODDIS', 'NOMDIS']]
        .drop_duplicates()
        .sort_values('NOMDIS')
        .rename(columns={'CODDIS': key_cod_dist, 'NOMDIS': key_nombre})
        .to_dict(orient='records')
    )   

    return dict_list

def build_lovs():
    
    lov_names = ["lov_district", "lov_date"]
    data_values = [build_districts_LOV(), build_dates_LOV()]
            
    lov_data_dict = [{"lov_name": lov_name, "data": data} for lov_name, data in zip(lov_names, data_values)]    
    
    return lov_data_dict


streamlit_lov_path = f'../source/data/LOV.json'
lovs = build_lovs()

with open(streamlit_lov_path, 'w') as json_file:
    json.dump(lovs, json_file)
